In [1]:
# Create a new and clear dataset
# The dataset contains only french companies as it is specified in the test statement (Ligne 5 in bold)
# In the new dataset, country_name and city_name are converted to real names that are exploited by the API provided in the test statement (used in the following cells)

# !pip install geopy

from geopy.geocoders import Nominatim
from geopy.distance import geodesic

from geopy.exc import GeocoderTimedOut

import csv
import json
import pandas as pd

print("HiHi")

dfLocations = pd.read_csv('locations.csv', sep=';')
dfCountry = pd.read_json('country.json')
dfCity = pd.read_json('city.json')

dfLocations.insert(3, "city", "_" , True)
dfLocations['city'] = dfLocations['city_id'].map(dfCity.set_index('id')['name'])
dfLocations.insert(2, "country", "_" , True)
dfLocations['country'] = dfLocations['country_id'].map(dfCountry.set_index('id')['name'])

IdFrance = dfCountry[(dfCountry['name'] == 'France')].values[0].tolist()[0]
subsetDataFrame = dfLocations[dfLocations['country_id'] == IdFrance]
filterinfDataframe = dfLocations[(dfLocations['is_headquarter'] == 1) & (dfLocations['country_id'] == IdFrance)]
Dtest =  filterinfDataframe[(filterinfDataframe['address'] .isnull())]
filterinfDataframe1 = filterinfDataframe.dropna()
fichierCSV = filterinfDataframe1.to_csv('Dataframe_result.csv', index=False, header=True)

# Function to extract longitude and latitude from an address
def longLatAddress(address):
    geolocator = Nominatim(user_agent="https://adresse.data.gouv.fr/api-doc/adresse")
    companyLocation = geolocator.geocode(address)

    if companyLocation is not None:
        companyLocationAddress = companyLocation.address
        companyLocationLatitude = companyLocation.latitude
        companyLocationlongitude = companyLocation.longitude
    else:
        companyLocationLatitude = None
        companyLocationlongitude = None
    return(companyLocationLatitude, companyLocationlongitude)

# Function to measure distance between a headquarter and its warehouses
def distanceLocations(headquarter, warehouse):
    distance = geodesic(headquarter, warehouse).kilometers
    if distance < 20:
        clas = "1"
    elif distance < 100 and distance > 20:
        clas = "2"
    elif distance > 100:
        clas = "3"
    return clas

# Function to extract headquarters from the new dataset clearLocations.csv
def extractHeadquarters(addressWarehouse):
    # Open the created dataset clearLocations.csv
    with open('Dataframe_result.csv') as csv_clearLocations:
        csvClearLocations = csv.reader(csv_clearLocations, delimiter=',')
        clearLocations = list(csvClearLocations)

    askedFile = open('askedFile.csv', 'w', encoding='utf-8')
    heading = "company_id,country_id,country,city_id,city,address,is_headquarter,class\n"
    askedFile.write(heading)
    warehouseLatitude, warehouselongitude = longLatAddress(addressWarehouse)
    lonLatWarehouse = (warehouseLatitude, warehouselongitude)
    for i in range(1, len(clearLocations)):
        headquarterLatitude, headquarterlongitude = longLatAddress(clearLocations[i][5] + " " + clearLocations[i][4] + " " + clearLocations[i][2])
        if headquarterLatitude is None and headquarterlongitude is None:
            row = "{},{},{},{},{},{},{},{}\n".format(clearLocations[i][0], clearLocations[i][1], clearLocations[i][2], clearLocations[i][3], clearLocations[i][4], clearLocations[i][5], clearLocations[i][6], "AddressIncorrect")
            askedFile.write(row)
        else:
            lonLatHeadquarter = (headquarterLatitude, headquarterlongitude)
            clas = distanceLocations(lonLatHeadquarter, lonLatWarehouse)
            row = "{},{},{},{},{},{},{},{}\n".format(clearLocations[i][0], clearLocations[i][1], clearLocations[i][2], clearLocations[i][3], clearLocations[i][4], clearLocations[i][5], clearLocations[i][6], clas)
            askedFile.write(row)
            askedFile.flush()

HiHi


In [2]:
# addressWarehouse = "16, Place de l'iris, Courbevoie, France"

addressWarehouse = "23, Rue des Jeûneurs, Paris, France"

extractHeadquarters(addressWarehouse)

In [ ]:
!pip install -r requirements.txt